In [1]:
import pandas as pd
import os
import re
from nltk.tokenize import TweetTokenizer
from nltk.stem import WordNetLemmatizer
from wordsegment import load, segment
import emoji
from preprocessing import process_single_tweet

In [2]:
training_data_file_path = "../data/OLIDv1/olid-training-v1.tsv"
train_data = pd.read_csv(training_data_file_path, sep='\t')
train_data.drop(['id', 'subtask_b', 'subtask_c'], axis=1, inplace=True)
print(train_data.shape)

(13240, 2)


In [3]:
train_data.head()

,tweet,subtask_a
0,@USER She should ask a few native Americans wh...,OFF
1,@USER @USER Go home you’re drunk!!! @USER #MAG...,OFF
2,Amazon is investigating Chinese employees who ...,NOT
3,"@USER Someone should'veTaken"" this piece of sh...",OFF
4,@USER @USER Obama wanted liberals &amp; illega...,NOT


Result from NLTK twitter tokenizer:

In [4]:
example = train_data["tweet"][1]
print(example)
twt_tokenizer = TweetTokenizer()
tokenized = twt_tokenizer.tokenize(example)
print(tokenized)

@USER @USER Go home you’re drunk!!! @USER #MAGA #Trump2020 👊🇺🇸👊 URL
['@USER', '@USER', 'Go', 'home', 'you', '’', 're', 'drunk', '!', '!', '!', '@USER', '#MAGA', '#Trump2020', '👊', '🇺', '🇸', '👊', 'URL']


Result from our tokenizer:

In [5]:
print(example)
print(process_single_tweet(example))

@USER @USER Go home you’re drunk!!! @USER #MAGA #Trump2020 👊🇺🇸👊 URL
['go', 'home', 'you', 'are', 'drunk', 'maga', 'trump', '2020', 'oncoming', 'fist', 'united', 'states', 'oncoming', 'fist']
